In [1]:
# Prosessing libs
import pandas as pd
import os
import numpy as np
from fastparquet import ParquetFile
import glob

# PySpark
from pyspark.sql import SparkSession
from pyspark.sql import functions as psf

# Visualisation libs
from matplotlib import pyplot as plt
import seaborn as sns
import plotly.express as px


In [2]:
# change memory allocation for pyarrow
# os.environ['PYARROW_MEMORY_POOL_MAX_CHUNKSIZE'] = '3000000000'
# os.environ['PYSPARK_DRIVER_PYTHON_OPTS'] = "notebook"
# os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable
# os.environ['PYSPARK_PYTHON'] = sys.executable

## Get the path of all files

In [3]:
os.getcwd()

'c:\\Users\\milan\\OneDrive - MUNI\\VŠ\\PhD\\Zahraniční stáž\\Work\\HeiGIT_notebooks\\analysis\\ai-assisted-osm-mapping-stats\\notebooks-contributions'

In [4]:
os.chdir("..")


In [5]:
home_dir = os.getcwd()
parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data-test")
parquet_dir

parquet_dir = r"E:\\Rafael-data\\type=way\\state=latest"


In [6]:
files = glob.glob(f'{parquet_dir}\\**\\*.parquet', recursive=True)
files


['E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_0.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_1.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_10.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_11.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_2.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_3.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_4.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_5.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_6.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_7.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_8.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2017\\data_9.parquet',
 'E:\\\\Rafael-data\\\\type=way\\\\state=latest\\year=2018\\data_0.parquet

In [7]:
# os.chdir("..")
# home_dir = os.getcwd()
# parquet_dir_node = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node")
# parquet_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data")
# duckdb_dir = os.path.join(home_dir, r"notebooks-contributions\duckdb")
# test_dir = os.path.join(home_dir, r"oshdb-contributions-parquet-data\type=node\year=2023")


In [8]:
# files = glob.glob(f'{parquet_dir}\\**\\*.parquet', recursive=True)
# files


In [9]:
# files_location_lst = []

# for (path_dir, folder_names, file_names) in os.walk(parquet_dir_node):
#     for file_name in file_names:
#         if file_name.endswith(".parquet"):
#             file_location = os.path.join(path_dir, file_name)
#             files_location_lst.append(file_location)
# files_location_lst


## Load the all data with Apache PySpark

In [10]:
spark = SparkSession.builder.appName("ReadParquetFiles").getOrCreate()

df = spark.read.parquet(*files)

# df.show()


In [11]:
# df.count()

In [12]:
# spark.stop()

In [13]:
# df.describe().show()

In [14]:
# df.na.drop(how="any",subset=["building_area"]).show()

### Data transformation, Data types conversion

In [11]:
# Cast the 'changeset_timestamp' column to a LongType
df = df.withColumn('changeset_timestamp', psf.col(
    'changeset_timestamp').cast('long'))

# Use the 'changeset_timestamp' column to create a new 'datetime' column
df = df.withColumn('datetime', psf.from_unixtime(
    psf.col('changeset_timestamp')/1000, 'yyyy-MM-dd HH:mm:ss.SSSSSS'))

# Show the resulting DataFrame
# df.show()


In [12]:
# Use the 'changeset_timestamp' column to create a new 'datetime' column
df = df.withColumn('y_m', psf.from_unixtime(
    psf.col('changeset_timestamp')/1000, 'yyyy-MM'))

# Create a new column called 'year' containing the year from the 'datetime' column
df = df.withColumn('year', psf.year('datetime'))



# Show the resulting DataFrame
# df.show()


In [13]:
# Convert the 'tags' column to a string for filtering
df = df.withColumn("tags_str", df.tags.cast("string"))
df = df.withColumn("road_str", df.road.cast("string"))

In [14]:
str(df.columns)

"['contrib_id', 'valid_from', 'valid_to', 'osm_id', 'osm_version', 'changeset_id', 'changeset_timestamp', 'hashtags', 'editor', 'user_id', 'tags', 'tags_delta', 'primary_feature', 'building', 'road', 'is_area', 'area', 'area_delta', 'length', 'length_delta', 'contrib_type', 'country_iso_a3', 'country', 'centroid', 'geometry_type', 'geometry_valid', 'geometry', 'state', 'type', 'datetime', 'y_m', 'year', 'tags_str', 'road_str']"

In [19]:
# df.count()

In [20]:
# df.groupBy("road").count().show()

In [21]:
# df.groupBy("state").count().show()

In [15]:
df = df.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [23]:
# df.groupBy("building").count().show()

In [24]:
df.columns

['contrib_id',
 'valid_from',
 'valid_to',
 'osm_id',
 'osm_version',
 'changeset_id',
 'changeset_timestamp',
 'hashtags',
 'editor',
 'user_id',
 'tags',
 'tags_delta',
 'primary_feature',
 'building',
 'road',
 'is_area',
 'area',
 'area_delta',
 'length',
 'length_delta',
 'contrib_type',
 'country_iso_a3',
 'country',
 'centroid',
 'geometry_type',
 'geometry_valid',
 'geometry',
 'state',
 'type',
 'datetime',
 'y_m',
 'year',
 'tags_str',
 'road_str',
 'hashtags_un']

In [25]:
# df.count()

# Number of created buildings

### Data wrangling – buildings
- select distinct osm_id
- remove all -1 and null values
- then use groupby (osm_id) and function last (time column)

In [12]:
# df.groupBy("building").count().show()

In [16]:
df_blds_created = df.filter(psf.col('building').isin([1])) # | psf.col('building').isNull()


In [14]:
# df_blds = df.filter(df.building.contains(1) | df.building.contains(0) | df.building.isNull())


In [15]:
# df_blds.show()

In [16]:
# df_blds.groupBy("building").count().show()


In [17]:
df_blds_created = df_blds_created.groupBy("osm_id", "editor", "country", "tags_str", "hashtags", "year").agg(psf.last("datetime").alias("datetime"))


In [18]:
# df_blds.show()

In [19]:
# df_blds.show()

In [18]:
df_blds_created = df_blds_created.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


## Building filter

### Filter for numbers of  created buildings
- column tag_str contains 'building' to filter all buildings

In [19]:
df_all_blds_created = df_blds_created.filter(df_blds_created.tags_str.contains("building"))


In [20]:
df_all_blds_created.groupBy("year").count().show()


+----+--------+
|year|   count|
+----+--------+
|2018|47926650|
|2023|17262951|
|2022|48635279|
|2019|34561719|
|2020|43439938|
|2017|37100023|
|2021|49210722|
+----+--------+



## AI created buildings filter

### Filter 1
- column tag_str contains 'microsoft/BuildingFootprints' or 'esri/Google_Africa_Buildings'

In [21]:
df_ai_blds_created = df_blds_created.filter(\
    df_blds_created.tags_str.contains("microsoft/BuildingFootprints")\
    | df_blds_created.tags_str.contains("esri/Google_Africa_Buildings")\
    )


In [22]:
df_ai_blds_created.groupBy("year").count().show()


+----+-------+
|year|  count|
+----+-------+
|2023|2810048|
|2022|6264764|
|2019|  10284|
|2020|2075573|
|2021|2770427|
+----+-------+



# Number of edited buildings

In [23]:
df_blds_edited = df.filter(psf.col('building').isin([0])) # | psf.col('building').isNull()


In [ ]:
# df_blds = df.filter(df.building.contains(1) | df.building.contains(0) | df.building.isNull())


In [ ]:
# df_blds.show()

In [ ]:
# df_blds.groupBy("building").count().show()


In [24]:
df_blds_edited = df_blds_edited.groupBy("osm_id", "editor", "country", "tags_str", "hashtags", "year").agg(psf.last("datetime").alias("datetime"))


In [ ]:
# df_blds.show()

In [ ]:
# df_blds.show()

In [25]:
df_blds_edited = df_blds_edited.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


## Building filter

### Filter for numbers of  edited buildings
- column tag_str contains 'building' to filter all buildings

In [26]:
df_all_blds_edited = df_blds_edited.filter(df_blds_edited.tags_str.contains("building"))


In [27]:
df_all_blds_edited.groupBy("year").count().show()


+----+--------+
|year|   count|
+----+--------+
|2018|11241012|
|2023| 9174288|
|2022|25258405|
|2019|15759176|
|2020|19295723|
|2017|10376007|
|2021|24667719|
+----+--------+



## AI edited buildings filter

### Filter 1
- column tag_str contains 'microsoft/BuildingFootprints' or 'esri/Google_Africa_Buildings'

In [28]:
df_ai_blds_edited = df_blds_edited.filter(\
    df_blds_edited.tags_str.contains("microsoft/BuildingFootprints")\
    | df_blds_edited.tags_str.contains("esri/Google_Africa_Buildings")\
    )


In [29]:
df_ai_blds_edited.groupBy("year").count().show()


+----+------+
|year| count|
+----+------+
|2023|456335|
|2022|904181|
|2019|    16|
|2020| 71203|
|2021|190505|
+----+------+



# Length of created roads

### Data wrangling – roads
- select distinct osm_id
- remove all -1
- then use groupby (osm_id) and function last (time column)

In [53]:
df_roads_created = df.filter(psf.col('road').isin([1]) | psf.col('road').isNull()) # | psf.col('road').isNull()

In [54]:
# df_roads = df.filter(~psf.col('road').isin([-1, 0]))

In [55]:
df_roads_created = df_roads_created.groupBy("osm_id", "editor", "length", "country", "tags_str", "hashtags", "year", "road").agg(psf.last("datetime").alias("datetime"))


In [56]:
# df_roads.show()

In [57]:
df_roads_created = df_roads_created.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [58]:
# df_roads.groupBy("hashtags_un").count().show()


In [59]:
# df_roads.show()

## Road filter

### Filter for the length of all roads
- column tag_str contains motorway, trunk, motorway_link, trunk_link, primary, primary_link, secondary, secondary_link, tertiary, tertiary_link, unclassified, residential

In [37]:
df_all_roads_created = df_roads_created.filter(
    df_roads_created.tags_str.contains("motorway")\
    | df_roads_created.tags_str.contains("trunk")\
    | df_roads_created.tags_str.contains("motorway_link")\
    | df_roads_created.tags_str.contains("trunk_link")\
    | df_roads_created.tags_str.contains("primary")\
    | df_roads_created.tags_str.contains("primary_link")\
    | df_roads_created.tags_str.contains("secondary")\
    | df_roads_created.tags_str.contains("secondary_link")\
    | df_roads_created.tags_str.contains("tertiary")\
    | df_roads_created.tags_str.contains("tertiary_link")\
    | df_roads_created.tags_str.contains("unclassified")\
    | df_roads_created.tags_str.contains("residential")\
    )


In [38]:
df_all_roads_created.groupBy("year").sum().show()


+----+----------------+-----------+-----------+---------+
|year|     sum(osm_id)|sum(length)|  sum(year)|sum(road)|
+----+----------------+-----------+-----------+---------+
|2018|1550223639544694|  845037963| 5310932040|  2631780|
|2023|1876143663291236|  537590431| 3426715194|  1693878|
|2022|4836994669745784| 1481303683| 9423648276|  4660558|
|2019|2296173607404444| 1061680562| 6729908472|  3333288|
|2020|4307910498440378| 1640580518|10834678040|  5363702|
|2017| 748685350028376|  587484763| 3027299164|  1500892|
|2021|5192803089722588| 1739022087|11296100602|  5589362|
+----+----------------+-----------+-----------+---------+



## AI road filter

### Filter 1
- column tags contains “#mapwithai” or “#nsroadimport” or "#MapWithAI"

In [39]:
df_roads_f1_created = df_roads_created.filter(\
    df_roads_created.tags_str.contains("mapwithai")\
    | df_roads_created.tags_str.contains("MapWithAI")\
    | df_roads_created.tags_str.contains("nsroadimport")\
    )


In [34]:
df_roads_f1_created.groupBy("year").sum().show()


+----+------------+-----------+---------+
|year| sum(osm_id)|sum(length)|sum(year)|
+----+------------+-----------+---------+
|2023|221644006172|     278590|   418761|
+----+------------+-----------+---------+



### Filter 2
- column tags_str contains (#nsroadimport" or "mapwithai" or “MapWithAI") and "highway"
OR
- column hashtags_un contains (#nsroadimport" or "mapwithai") and it is highway (from column tags_str ???)

In [35]:
df_roads_f2_created = df_roads_created.filter(\
    (
        (df_roads_created.tags_str.contains("mapwithai")\
        | df_roads_created.tags_str.contains("MapWithAI")\
        | df_roads_created.tags_str.contains("nsroadimport"))\
        & (df_roads_created.tags_str.contains("highway")))\
        |\
        (df_roads_created.hashtags_un.contains("mapwithai")\
        | df_roads_created.hashtags_un.contains("nsroadimport")\
        & (df_roads_created.tags_str.contains("highway")))\
    )


In [36]:
df_roads_f2_created.groupBy("year").sum().show()


+----+---------------+-----------+---------+
|year|    sum(osm_id)|sum(length)|sum(year)|
+----+---------------+-----------+---------+
|2023|165903613475359|  243608349|367680250|
+----+---------------+-----------+---------+



### Filter 3
3_1 – column editor contains RapiD

3_2 – column editor contains RapiD or column tags_str/hashtags_un contains mapwithai

3_3 – column editor contains RapiD and column tags_str/hashtags_un contains mapwithai

#### F3_1

In [37]:
df_roads_f3_1_created = df_roads_created.filter(df_roads_created.editor.contains("RapiD"))


In [38]:
df_roads_f3_1_created.groupBy("year").sum().show()


+----+---------------+-----------+---------+
|year|    sum(osm_id)|sum(length)|sum(year)|
+----+---------------+-----------+---------+
|2023|155386313768669|  194174637|389579225|
+----+---------------+-----------+---------+



#### F3_2

In [60]:
df_roads_f3_2_created = df_roads_created.filter(
    df_roads_created.editor.contains("RapiD")\
    | df_roads_created.tags_str.contains("mapwithai")\
    | df_roads_created.tags_str.contains("MapWithAI")\
    | df_roads_created.tags_str.contains("nsroadimport")\
    | df_roads_created.hashtags_un.contains("#mapwithai")\
    | df_roads_created.hashtags_un.contains("#nsroadimport")\
    | df_roads_created.hashtags_un.contains("#MapWithAI")\
)


In [61]:
df_roads_f3_2_created.groupBy("year").sum().show()


+----+----------------+-----------+-----------+---------+
|year|     sum(osm_id)|sum(length)|  sum(year)|sum(road)|
+----+----------------+-----------+-----------+---------+
|2018| 103751512284200|   58900549|  358655104|   107491|
|2023|4089982442760974|  233935088| 7457233175|   195445|
|2022|7353034221225215|  674568081|14382079578|  1115271|
|2019| 406493157010013|   91166802| 1137621702|   113269|
|2020|4199113604817773|  781642857|10568438000|  1023906|
|2017|  62286114531654|   36559347|  243308693|    85009|
|2021|5732717372847825|  861096352|12610732808|  1339105|
+----+----------------+-----------+-----------+---------+



#### F3_3

In [41]:
df_roads_f3_3_created = df_roads_created.filter(\
    (df_roads_created.editor.contains("RapiD")\
     & (df_roads_created.tags_str.contains("mapwithai")\
        | df_roads_created.hashtags_un.contains("mapwithai")\
        | df_roads_created.tags_str.contains("MapWithAI")\
        | df_roads_created.hashtags_un.contains("MapWithAI")))\
    )


In [42]:
df_roads_f3_3_created.groupBy("year").sum().show()


+----+--------------+-----------+---------+
|year|   sum(osm_id)|sum(length)|sum(year)|
+----+--------------+-----------+---------+
|2023|52019131287119|   65803042|101700256|
+----+--------------+-----------+---------+



# Length of edited roads

In [62]:
df_roads_edited = df.filter(psf.col('road').isin([0]) | psf.col('road').isNull()) #  | psf.col('road').isNull()

In [63]:
# df_roads = df.filter(~psf.col('road').isin([-1, 0]))

In [64]:
df_roads_edited = df_roads_edited.groupBy("osm_id", "editor", "length", "country", "tags_str", "hashtags", "year", "road").agg(psf.last("datetime").alias("datetime"))


In [65]:
# df_roads.show()

In [66]:
df_roads_edited = df_roads_edited.withColumn("hashtags_un", psf.concat_ws(",", psf.col("hashtags")))


In [67]:
# df_roads.groupBy("hashtags_un").count().show()


In [68]:
# df_roads.show()

## Edited roads filter

### Filter for the length of all edited roads
- column tag_str contains motorway, trunk, motorway_link, trunk_link, primary, primary_link, secondary, secondary_link, tertiary, tertiary_link, unclassified, residential

In [49]:
df_all_roads_edited = df_roads_edited.filter(
    df_roads_edited.tags_str.contains("motorway")\
    | df_roads_edited.tags_str.contains("trunk")\
    | df_roads_edited.tags_str.contains("motorway_link")\
    | df_roads_edited.tags_str.contains("trunk_link")\
    | df_roads_edited.tags_str.contains("primary")\
    | df_roads_edited.tags_str.contains("primary_link")\
    | df_roads_edited.tags_str.contains("secondary")\
    | df_roads_edited.tags_str.contains("secondary_link")\
    | df_roads_edited.tags_str.contains("tertiary")\
    | df_roads_edited.tags_str.contains("tertiary_link")\
    | df_roads_edited.tags_str.contains("unclassified")\
    | df_roads_edited.tags_str.contains("residential")\
    )


In [50]:
df_all_roads_edited.groupBy("year").sum().show()


+----+----------------+-----------+-----------+---------+
|year|     sum(osm_id)|sum(length)|  sum(year)|sum(road)|
+----+----------------+-----------+-----------+---------+
|2018|1279381789094765| 2071830049| 8994167478|        0|
|2023|3387588511203951| 4981938677|16432754149|        0|
|2022|6702389044966362|10107825741|32573123898|        0|
|2019|2359189383749070| 3364445095|13583282832|        0|
|2020|5027496457881478| 6089878324|23814808180|        0|
|2017| 470707805169851| 1182720278| 5021807597|        0|
|2021|6330211301299034| 8973919759|29461808577|        0|
+----+----------------+-----------+-----------+---------+



## AI road filter

### Filter 1
- column tags contains “#mapwithai” or “#nsroadimport” or "#MapWithAI"

In [ ]:
df_roads_f1_edited = df_roads_edited.filter(\
    df_roads_edited.tags_str.contains("mapwithai")\
    | df_roads_edited.tags_str.contains("MapWithAI")\
    | df_roads_edited.tags_str.contains("nsroadimport")\
    )


In [ ]:
df_roads_f1_edited.groupBy("year").sum().show()


+----+------------+-----------+---------+
|year| sum(osm_id)|sum(length)|sum(year)|
+----+------------+-----------+---------+
|2023|221644006172|     278590|   418761|
+----+------------+-----------+---------+



### Filter 2
- column tags_str contains (#nsroadimport" or "mapwithai" or “MapWithAI") and "highway"
OR
- column hashtags_un contains (#nsroadimport" or "mapwithai") and it is highway (from column tags_str ???)

In [ ]:
df_roads_f2_edited = df_roads_edited.filter(\
    (\
        (df_roads_edited.tags_str.contains("mapwithai")\
         | df_roads_edited.tags_str.contains("MapWithAI")\
         | df_roads_edited.tags_str.contains("nsroadimport"))\
        & (df_roads_edited.tags_str.contains("highway")))\
      |\
    (df_roads_edited.hashtags_un.contains("mapwithai")\
         | df_roads_edited.hashtags_un.contains("nsroadimport")\
         & (df_roads_edited.tags_str.contains("highway"))))


In [ ]:
df_roads_f2_edited.groupBy("year").sum().show()


+----+---------------+-----------+---------+
|year|    sum(osm_id)|sum(length)|sum(year)|
+----+---------------+-----------+---------+
|2023|165903613475359|  243608349|367680250|
+----+---------------+-----------+---------+



### Filter 3
3_1 – column editor contains RapiD

3_2 – column editor contains RapiD or column tags_str/hashtags_un contains mapwithai

3_3 – column editor contains RapiD and column tags_str/hashtags_un contains mapwithai

#### F3_1

In [ ]:
df_roads_f3_1_edited = df_roads_edited.filter(df_roads_edited.editor.contains("RapiD"))


In [ ]:
df_roads_f3_1_edited.groupBy("year").sum().show()


+----+---------------+-----------+---------+
|year|    sum(osm_id)|sum(length)|sum(year)|
+----+---------------+-----------+---------+
|2023|155386313768669|  194174637|389579225|
+----+---------------+-----------+---------+



#### F3_2

In [69]:
df_roads_f3_2_edited = df_roads_edited.filter(\
    df_roads_edited.editor.contains("RapiD")\
    | df_roads_edited.tags_str.contains("mapwithai")\
    | df_roads_edited.tags_str.contains("MapWithAI")\
    | df_roads_edited.tags_str.contains("nsroadimport")\
    | df_roads_edited.hashtags_un.contains("#mapwithai")\
    | df_roads_edited.hashtags_un.contains("#nsroadimport")\
    | df_roads_edited.hashtags_un.contains("#MapWithAI")\
)


In [70]:
df_roads_f3_2_edited.groupBy("year").sum().show()


+----+----------------+-----------+-----------+---------+
|year|     sum(osm_id)|sum(length)|  sum(year)|sum(road)|
+----+----------------+-----------+-----------+---------+
|2018|  47831349451073|   58785928|  195604740|        0|
|2023|4009166160789226|  342576462| 7461784925|        0|
|2022|6858235782523571| 1019141248|14095831104|        0|
|2019| 339800777674706|   82567473|  972243393|        0|
|2020|3833202295517011|  981620273|10205409660|        0|
|2017|  21546425517552|   29790191|   97215366|        0|
|2021|5214391908479843| 1345649228|12086976511|        0|
+----+----------------+-----------+-----------+---------+



#### F3_3

In [ ]:
df_roads_f3_3_edited = df_roads_edited.filter(\
    (df_roads_edited.editor.contains("RapiD")\
     & (df_roads_edited.tags_str.contains("mapwithai")\
        | df_roads_edited.hashtags_un.contains("mapwithai")\
        | df_roads_edited.tags_str.contains("MapWithAI")\
        | df_roads_edited.hashtags_un.contains("MapWithAI")))\
    )


In [ ]:
df_roads_f3_3_edited.groupBy("year").sum().show()


+----+--------------+-----------+---------+
|year|   sum(osm_id)|sum(length)|sum(year)|
+----+--------------+-----------+---------+
|2023|52019131287119|   65803042|101700256|
+----+--------------+-----------+---------+



# This chapter is useless so far

## Convert to Pandas DataFrame - AI buildings, AI roads (this chapter is useless so far)

In [25]:
# Filter AI blds
df_blds = df.filter(df.tags_str.contains("microsoft/BuildingFootprints") | df.tags_str.contains("esri/Google_Africa_Buildings"))\
.groupby("country", "editor", "year", "tags_str")\
.agg(psf.sum("user_id"))\
.withColumnRenamed("sum(user_id)", "count_u_id")\
.sort(psf.col("count_u_id").desc())\
.toPandas()


In [26]:
df_blds

,country,editor,year,tags_str,count_u_id
0,TUR,RapiD 1.1.9-tm.1,2023,"{building -> yes, source -> microsoft/Building...",1862607831910
1,IND,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",697575610154
2,IDN,JOSM/1.5 (18583 en),2023,"{building -> yes, source -> microsoft/Building...",167757823842
3,USA,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",116856923771
4,SYR,RapiD 1.1.9-tm.1,2023,"{building -> yes, source -> microsoft/Building...",97647827025
...,...,...,...,...,...
24984,GBR,StreetComplete 50.2,2023,"{building -> house, source -> microsoft/Buildi...",7329
24985,GBR,StreetComplete 50.2,2023,"{building -> detached, source -> microsoft/Bui...",7329
24986,GBR,StreetComplete 50.2,2023,"{building -> apartments, source -> microsoft/B...",7329
24987,GBR,RapiD 1.1.9,2023,"{building -> yes, source -> microsoft/Building...",7329


In [32]:
df_blds.notna().all(axis=1)


0        True
1        True
2        True
3        True
4        True
         ... 
24984    True
24985    True
24986    True
24987    True
24988    True
Length: 24989, dtype: bool

In [ ]:
# & (psf.col("road") == 1)


In [98]:
# Filter AI roads
df_roads = df_roads.filter(((df_roads.tags_str.contains("mapwithai") | df_roads.tags_str.contains("MapWithAI") | df_roads.tags_str.contains("#nsroadimport")) & (df_roads.tags_str.contains("highway")))
                           | (df_roads.hashtags_un.contains("#mapwithai") | df_roads.hashtags_un.contains("#nsroadimport")))\
    .groupby("country", "editor", "year", "tags_str", "hashtags_un", "length")\
    .agg(psf.sum("country"))\
    .withColumnRenamed("sum(country)", "count_country")\
    .sort(psf.col("count_country").desc())\
    .toPandas()


ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "c:\Users\milan\general_py_venv\lib\site-packages\py4j\java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "c:\Users\milan\general_py_venv\lib\site-packages\py4j\clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "C:\Users\milan\AppData\Local\Programs\Python\Python310\lib\socket.py", line 705, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
df_roads


,country,editor,year,tags_str,hashtags_un,length,count_osm_id
0,TZA,JOSM/1.5 (18646 en),2023,{},"#Tanzania,#mapwithai",0,3883303105856
1,TZA,JOSM/1.5 (18583 en),2023,{},"#Tanzania,#mapwithai",0,2204724899226
2,IND,RapiD 1.1.9-tm.1,2023,{},"#India,#mapwithai",0,1786006015589
3,IND,JOSM/1.5 (18427 en),2023,{},"#India,#mapwithai",0,1006969338360
4,TZA,JOSM/1.5 (18463 en),2023,{},"#Tanzania,#mapwithai",0,841542487332
...,...,...,...,...,...,...,...
82556,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",329,13331490
82557,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",340,13303745
82558,USA,JOSM/1.5 (18700 en),2023,"{highway -> residential, tiger:county -> Orang...","#buildingmapping,#mapwithai",399,13303745
82559,USA,JOSM/1.5 (18565 en),2023,"{highway -> tertiary, tiger:county -> Orange, ...","#buildingmapping,#mapwithai",697,13300336
